In [ ]:
!git clone https://github.com/threestudio-project/threestudio.git

In [ ]:
!sudo apt-get update
!sudo apt-get install mesa-common-dev libegl1-mesa-dev -y
!sudo apt-get install ninja-build  -y

In [ ]:
%pip install --upgrade pip

In [ ]:
%cd /notebooks/threestudio

In [ ]:
!!git pull

In [ ]:
""" %pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113 """
%pip install ninja
%pip install  torch==2.0.0+cu118 torchvision==0.15.1+cu118 --index-url https://download.pytorch.org/whl/cu118 
%pip install mediapipe


In [ ]:
%pip install  -r requirements.txt  --upgrade  # --force-reinstall

In [ ]:
%pip install lightning==2.0.0 omegaconf==2.3.0 jaxtyping typeguard diffusers transformers accelerate opencv-python tensorboard matplotlib imageio imageio[ffmpeg] trimesh bitsandbytes sentencepiece safetensors huggingface_hub libigl xatlas 
%pip install open3d plotly # mesh visualization

In [ ]:
%pip install git+https://github.com/KAIR-BAIR/nerfacc.git@v0.5.2
%pip install git+https://github.com/NVlabs/nvdiffrast.git

In [ ]:
%pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

In [ ]:
!mkdir load/zero123
%cd load/zero123
!wget https://zero123.cs.columbia.edu/assets/zero123-xl.ckpt
%cd ../../

In [ ]:
!rm -rf ./outputs

In [ ]:
import os
%env CUDA_VISIBLE_DEVICES=0
# Set environment variable
name = "dragon3"
prompt = "an 3D render of a cartoon skeleton of a chibi dragon, detailed, 8k"
last="last-v3"
%cd /notebooks/threestudio

In [ ]:
""" data.default_elevation_deg=0
data.default_azimuth_deg=90
data.default_camera_distance=1
system.freq.guidance_eval=0 """
cmd = f''' --train --gpu 0
--config configs/zero123_64.yaml
data.image_path="load/images/{name}_rgba.png"
name="{name}"
tag=Phase1
system.loggers.wandb.enable=false
use_timestamp=False
data.default_azimuth_deg=0
data.random_camera.batch_size=14
system.loss.lambda_depth=0.05
system.loss.lambda_depth_rel="[0, 0, 0.05, 100]"
system.loss.lambda_normal="[0, 0, 0.05, 100]"
system.geometry.density_blob_scale=8.
system.freq.guidance_eval=37
system.guidance.cond_elevation_deg=0
'''

%run launch.py {cmd}

In [ ]:
# Phase 1.5 - 512 refine
#system.loggers.wandb.project="zero123" system.loggers.wandb.name=${NAME}_Phase1p5

cmd = f'''--config configs/zero123-geometry.yaml --train --gpu 0 
data.image_path=./load/images/{name}_rgba.png 
system.geometry_convert_from=./outputs/{name}/Phase1/ckpts/{last}.ckpt 
use_timestamp=False name={name} tag=Phase1p5'''

%run launch.py {cmd}

In [ ]:
# Phase 2 - dreamfusion
#data.random_camera.batch_size=2 trainer.accumulate_grad_batches=4 system.freq.guidance_eval=0
# system.freq.guidance_eval=0 system.loggers.wandb.enable=false system.loggers.wandb.project="zero123" system.loggers.wandb.name=${NAME}_Phase2
tag="Phase1p5"
last="last"
cmd = f'''--config configs/experimental/imagecondition_zero123nerf.yaml 
--train --gpu 0 data.image_path=./load/images/{name}_rgba.png
system.prompt_processor.prompt="{prompt}" 
system.weights="./outputs/{name}/{tag}/ckpts/{last}.ckpt" 
system.freq.guidance_eval=23
name={name} tag=Phase2 system.loggers.wandb.enable=false  use_timestamp=False'''

%run launch.py {cmd}

In [ ]:

# Phase 2 - dreamfusion
cmd = f'''--config configs/experimental/my_imagecondition_zero123nerf_refine.yaml 
--train --gpu 0  system.prompt_processor.prompt="{prompt}" 
system.geometry_convert_from="./outputs/{name}/{phase}/ckpts/{last}.ckpt" 
name={name} 
tag=Phase2_refine system.loggers.wandb.enable=false 
use_timestamp=False
data.image_path=./load/images/{name}_rgba.png
'''

%run launch.py {cmd}

In [ ]:
tag="Phase1p5"
last="last"
cmd=f'''--config "outputs/{name}/{tag}/configs/parsed.yaml" 
--export --gpu 0 resume="outputs/{name}/{tag}/ckpts/{last}.ckpt" 
system.exporter.context_type=cuda  system.exporter_type=mesh-exporter'''
%run launch.py {cmd}

TEXT MESH

In [ ]:
# uses DeepFloyd IF, requires ~15GB VRAM
cmd= f'''
--config configs/textmesh-if.yaml --train --gpu 0 tag=DRAGON system.prompt_processor.prompt="{prompt}"
system.prompt_processor_type="stable-diffusion-prompt-processor"
system.guidance_type="stable-diffusion-guidance"
system.guidance.pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1"
system.prompt_processor.pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1"
'''
%run launch.py {cmd}

In [ ]:
tag="DRAGON@20230726-085910"
name="textmesh-if"
cmd=f'--config "outputs/{name}/{tag}/configs/parsed.yaml" --export --gpu 0 resume="outputs/{name}/{tag}/ckpts/last.ckpt" system.exporter.context_type=cuda  system.exporter_type=mesh-exporter'
%run launch.py {cmd}

CONTROL 4D

In [ ]:
cmd = f'''--config configs/control4d-static.yaml --train --gpu 0 
data.dataroot="YOUR_DATAROOT/twindom" 
system.prompt_processor.prompt="{prompt}" 
name={name} 
tag=Phase1 
system.loggers.wandb.enable=false 
use_timestamp=False'''

%run launch.py {cmd}

OTHER

NERF STUDIO

pip install 'mediapipe'
sudo apt-get update && apt-get install -y --no-install-recommends \
    pkg-config \
    libglvnd0 \
    libgl1 \
    libglx0 \
    libegl1 \
    libgles2 \
    libglvnd-dev \
    libgl1-mesa-dev \
    libegl1-mesa-dev \
    libgles2-mesa-dev \
    cmake \
    curl \
    libsm6 \
    libxext6 \
    libxrender-dev
sudo apt-get install libnvidia-gl-418-server
sudo apt-get install  libnvidia-gl 
sudo apt-get update
sudo apt-get install mesa-common-dev libegl1-mesa-dev
sudo apt-get install ninja-build
source myenv/bin/activate
deactivate
python3.10 -m ipykernel install  --name=myenv
python3.10 launch.py --config "outputs/zero123/[64, 128, 256]_front_rgba.png_prog0@20230716-010647/configs/parsed.yaml" --export --gpu 0 resume="outputs/zero123/[64, 128, 256]_front_rgba.png_prog0@20230716-010647/ckpts/last.ckpt" system.exporter_type=mesh-exporter

sudo apt install colmap

NERF STUDIO

In [ ]:
!sudo apt-get -y install \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev

In [ ]:
!sudo apt-get  install -y '^libxcb.*-dev' libx11-xcb-dev libglu1-mesa-dev libxrender-dev libxi-dev libxkbcommon-dev libxkbcommon-x11-dev

In [ ]:
!sudo apt-get install -y  libxcb-xinerama0

In [ ]:
!sudo apt-get update && apt-get install -y --no-install-recommends \
    pkg-config \
    libglvnd0 \
    libgl1 \
    libglx0 \
    libegl1 \
    libgles2 \
    libglvnd-dev \
    libgl1-mesa-dev \
    libegl1-mesa-dev \
    libgles2-mesa-dev \
    cmake \
    curl \
    libsm6 \
    libxext6 \
    libxrender-dev

In [ ]:
%cd ../

In [ ]:
!git clone https://github.com/nerfstudio-project/nerfstudio.git


In [ ]:
%cd nerfstudio

In [ ]:

%pip install  --upgrade pip setuptools
%pip install -e . --force-reinstall

In [ ]:
%cd ../

In [ ]:
!git clone https://github.com/colmap/colmap

In [ ]:
!sudo apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev

In [ ]:
!sudo apt-get install -y \
    nvidia-cuda-toolkit \
    nvidia-cuda-toolkit-gcc

In [ ]:
!sudo apt-get install -y libgtest-dev

In [ ]:
%cd colmap
!git checkout master
!mkdir build
%cd build
!cmake .. -GNinja
!ninja
!sudo ninja install

In [ ]:
%pip uninstall opencv-python
%pip install  opencv-python-headless --force-reinstall

In [ ]:
%pip install --upgrade pip
%pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118


In [ ]:
%pip install nerfstudio

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
!/notebooks/myenv/bin/ns-process-data video --data /notebooks/dragon-v/dragon.mkv --output-dir /notebooks/dragon-v/colmap 

In [ ]:

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
print(websocket_url)
!/notebooks/myenv/bin/ns-train nerfacto --data /notebooks/robot 


In [ ]:
# Make the directory if it doesn't exist yet.
# This location is recommended by the distribution maintainers.
!sudo mkdir --parents --mode=0755 /etc/apt/keyrings
# Download the key, convert the signing-key to a full
# keyring required by apt and store in the keyring directory
!wget https://repo.radeon.com/rocm/rocm.gpg.key -O - | \
    gpg --dearmor | sudo tee /etc/apt/keyrings/rocm.gpg > /dev/null

In [ ]:
# Kernel driver repository for jammy
!sudo tee /etc/apt/sources.list.d/amdgpu.list <<'EOF'
!deb [arch=amd64 signed-by=/etc/apt/keyrings/rocm.gpg] https://repo.radeon.com/amdgpu/latest/ubuntu jammy main
!EOF
# ROCm repository for jammy
!sudo tee /etc/apt/sources.list.d/rocm.list <<'EOF'
!deb [arch=amd64 signed-by=/etc/apt/keyrings/rocm.gpg] https://repo.radeon.com/rocm/apt/debian jammy main
!EOF
# Prefer packages from the rocm repository over system packages
!echo -e 'Package: *\nPin: release o=repo.radeon.com\nPin-Priority: 600' | sudo tee /etc/apt/preferences.d/rocm-pin-600

In [ ]:
!sudo apt install -y amdgpu-dkms

In [ ]:

#%pip install --ignore-installed torchaudio==0.10.0+cu113

In [ ]:
%pip install torch==2.0.1+cu118 torchaudio torchvision+cu118 --force-reinstall -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
%pip install   torchaudio==2.0.2+cu118 torchvision==0.15.2+cu118 torch==2.0.1+cu118 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!/notebooks/myenv/bin/ns-export tsdf --load-config outputs/robot/nerfacto/2023-07-18_094132/config.yml    --output-dir ./robot-nerfacto-tsdf

In [ ]:
!ns-download-data nerfstudio --capture-name=poster